# Climatology Functions

```{warning} This is not meant to be a standalone notebook.
This notebook is part of the process we have for adding entries to the NCL Index and is not meant to be used as tutorial or example code.
```

## Functions covered

- [calcDayAnomTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/calcDayAnomTLL.shtml)
- [calcMonAnomTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/calcMonAnomTLL.shtml)
- [clmDayTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/clmDayTLL.shtml)
- [clmMonTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/clmMonTLL.shtml)
- [month_to_season](https://www.ncl.ucar.edu/Document/Functions/Contributed/month_to_season.shtml)
- [rmMonAnnCycTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/rmMonAnnCycTLL.shtml)
- [stdMonTLL](https://www.ncl.ucar.edu/Document/Functions/Contributed/stdMonTLL.shtml)

## NCL code

```{literalinclude} ../ncl_raw/climatology.ncl

```

## Python Functionality

In [ ]:
import math
import xarray as xr
import geocat.datafiles as gcd
from geocat.comp import month_to_season

results = {}

ds_m = xr.open_dataset(
    gcd.get("applications_files/inputs/CMIP6_sea_ice_monthly_subset.nc")
)
ds_d = xr.open_dataset(
    gcd.get("applications_files/inputs/CMIP6_sea_ice_daily_subset.nc")
)

# calcDayAnomTLL
aice = ds_d.aice_d[:, 10, 10]
DayTLL = aice.groupby(aice.time.dt.dayofyear)
clmDayTLL = DayTLL.mean(dim="time")
calcDayAnomTLL = DayTLL - clmDayTLL
results["calcDayAnomTLL"] = calcDayAnomTLL.item(0)

# calcMonAnomTLL
aice = ds_m.aice[:, 10, 10]
MonTLL = aice.groupby(aice.time.dt.month)
clmMonTLL = MonTLL.mean(dim="time")
calcMonAnomTLL = MonTLL - clmMonTLL
results["calcMonAnomTLL"] = calcMonAnomTLL.item(0)

# clmDayTLL
aice = ds_d.aice_d[:, 10, 10]
DayTLL = aice.groupby(aice.time.dt.dayofyear)
clmDayTLL = DayTLL.mean(dim="time")
results["clmDayTLL"] = clmDayTLL.item(0)

# clmMonTLL
aice = ds_m.aice[:, 10, 10]
MonTLL = aice.groupby(aice.time.dt.month)
clmMonTLL = MonTLL.mean(dim="time")
results["clmMonTLL"] = clmMonTLL.item(0)

# month_to_season
aice = ds_m.aice[:, 10, 10]
mon_to_season = month_to_season(aice, "ASO")
results["month_to_season"] = mon_to_season.item(0)

# rmMonAnnCycTLL
aice = ds_m.aice[:, 10, 10]
MonTLL = aice.groupby(aice.time.dt.month)
clmMonTLL = MonTLL.mean(dim="time")
rmMonAnnCycTLL = MonTLL - clmMonTLL
results["rmMonAnnCycTLL"] = rmMonAnnCycTLL.item(0)

# stdMonTLL
aice = ds_m.aice[:, 10, 10]
MonTLL = aice.groupby(aice.time.dt.month)
stdMonTLL = MonTLL.std(ddof=1)
results["stdMonTLL"] = stdMonTLL.item(0)

## Comparison

In [ ]:
# dictionary of climatology functions results
ncl_results = {
    "calcDayAnomTLL": -0.02774364,
    "calcMonAnomTLL": 0.003319263,
    "clmDayTLL": 0.7408184,
    "clmMonTLL": 0.8063018,
    "month_to_season": 0.9931215,
    "rmMonAnnCycTLL": 0.003319263,
    "stdMonTLL": 0.07392025,
}

for c in ncl_results.keys() & results.keys():
    print(f"{c}: \n\tpython:\t\t{results[c]}\n\tncl:\t\t{ncl_results[c]}\n")
    assert math.isclose(ncl_results[c], results[c], rel_tol=1e-06, abs_tol=1e-06)

## Differences

In [ ]:
for c in ncl_results.keys() & results.keys():
    print(f"{c}:")
    print(f"\t{results[c] - ncl_results[c]}")